<a href="https://colab.research.google.com/github/mmobik/Practices/blob/main/%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_0_8_0_%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests


def main():
    print("Добро пожаловать в панель управления!\n")
    print("Список доступных функций:")
    print("1. Вывод картинки дня (APOD)")
    print("2. Вывод фотографий с марсохода")
    print("3. Поиск и отображение информации об объектах, сближающихся с Землей")
    print("4. Вывод данных о космической погоде")
    print("5. Выход\n")

    while True:
        user = int(input("Пожалуйста введите номер функции: "))
        if user == 1:
            apod()  # Вывод картинки дня
        elif user == 2:
            mars_rover()  # Вывод фотографий с марсохода
        elif user == 3:
            neows()  # Поиск информации об астероидах
        elif user == 4:
            cosmo_weather()  # Вывод данных о космической погоде
        elif user == 5:
            break  # Выход из программы
        else:
            print("Номер функции не найден, повторите свой запрос!")


def apod():
    api_key = "bG5oVfagxzZvMa90zbikCxCvIwG0dXzu8AGNuMdJ"
    url = f"https://api.nasa.gov/planetary/apod?api_key={api_key}"

    # Запрос данных о картинке дня
    response = requests.get(url).json()

    print(f"\nЗаголовок: {response['title']}")
    print(f"Дата: {response['date']}")
    print(f"Ссылка на изображение: {response['hdurl']}")


def mars_rover():
    api_key = "bG5oVfagxzZvMa90zbikCxCvIwG0dXzu8AGNuMdJ"
    status_url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity?api_key={api_key}"

    # Запрос статуса марсохода
    status_response = requests.get(status_url)

    if status_response.status_code == 200:
        status_response = status_response.json()
        print(f"\nСтатус марсохода: {status_response['rover']['status']}")
        print(f"Самая последняя дата фото: {status_response['rover']['max_date']}")
        print(f"Количество фото: {status_response['rover']['total_photos']}")

        # Вывод доступных камер
        cameras = [camera["name"] for camera in status_response['rover']['cameras']]
        print(f"Список доступных камер: {cameras}")
    else:
        print(f"Ошибка запроса {status_response.status_code}")

    # Ввод данных для поиска фотографий
    camera = input("Введите камеру: ").lower()
    yr = int(input("Введите год: "))
    month = int(input("Введите месяц: "))
    day = int(input("Введите день: "))

    img_url = []
    url = (f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={yr}"
           f"-{month}-{day}&api_key={api_key}")

    # Запрос фотографий
    response = requests.get(url)
    if response.status_code == 200:
        response = response.json()
        photos = response["photos"]

        # Вывод фотографий для указанной камеры
        if photos:
            for photo in photos:
                if photo["camera"]["name"].lower() == camera:
                    img_url.append(photo["img_src"])

            for count, photo in enumerate(img_url, start=1):
                print(f"Ссылка на {count} фото: {photo}")
        else:
            print("Нет фотографий для этой даты.")
    else:
        print(f"Ошибка запроса {response.status_code}")


def neows():
    api_key = "bG5oVfagxzZvMa90zbikCxCvIwG0dXzu8AGNuMdJ"
    start_date = input("Введите дату начала поиска астероидов в формате ГГГГ-ММ-ДД: ")
    end_date = input("Введите дату конца поиска астероидов (разница не более 7 дней) в формате ГГГГ-ММ-ДД: ")

    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}"

    # Запрос данных об астероидах
    response = requests.get(url)
    if response.status_code == 200:
        response = response.json()
        print(f"Количество найденных астероидов: {response['element_count']}")

        # Вывод информации об астероидах
        for count, data in enumerate(response['near_earth_objects'][start_date], start=1):
            print(f"Имя №{count} астероида: {data['name']},"
                  f" размеры {(data['estimated_diameter']['kilometers']['estimated_diameter_min']):.2f}"
                  f"-{(data['estimated_diameter']['kilometers']['estimated_diameter_max']):.2f} км,"
                  f" опасность: {data['is_potentially_hazardous_asteroid']}")
    else:
        print(f"Ошибка номер {response.status_code}")


def cosmo_weather():
    while True:
        start_date = input("Введите диапазон вида: 'ГГГГ ММ ДД' до текущей даты включительно: ").split()
        if int(start_date[1]) > 12 or int(start_date[2]) > 31:
            print("Неправильный ввод даты.")
            continue
        else:
            break

    api_key = "bG5oVfagxzZvMa90zbikCxCvIwG0dXzu8AGNuMdJ"

    # URL для солнечных вспышек и геомагнитных бурь
    sun_url = (f"https://api.nasa.gov/DONKI/FLR?startDate={start_date[0]}-{start_date[1]}-{start_date[2]}"
               f"&endDate=2024-11-16&api_key={api_key}")

    storm_url = (f"https://api.nasa.gov/DONKI/GST?startDate={start_date[0]}-{start_date[1]}-{start_date[2]}"
                 f"&endDate=2024-11-16&api_key={api_key}")

    # Запрос данных о солнечных вспышках
    sun_response = requests.get(sun_url)
    storm_response = requests.get(storm_url)

    if sun_response.status_code == 200 and storm_response.status_code == 200:
        sun_response = sun_response.json()
        if sun_response:
            print(f"Данные о солнечных вспышках:\n")
            for flare in sun_response:
                print(f"Название вспышки: {flare['flrID']}, Начало: {flare['beginTime']}, Пик: {flare['peakTime']},"
                      f" Конец: {flare['endTime']}")
        else:
            print("Данные о солнечных вспышках не найдены.")

        storm_response = storm_response.json()
        if storm_response:
            print(f"Данные о геомагнитных бурях:\n")
            for storm in storm_response:
                print(f"Название бури: {storm['gstID']}")
                for kp in storm['allKpIndex']:
                    print(f"Дата: {kp['observedTime']}, активность {kp['kpIndex']}")
        else:
            print("Данные о геомагнитных бурях не найдены.")
    else:
        print(f"Ошибка номер {sun_response.status_code}")


# Запуск основной функции
main()


Добро пожаловать в панель управления!

Список доступных функций:
1. Вывод картинки дня (APOD)
2. Вывод фотографий с марсохода
3. Поиск и отображение информации об объектах, сближающихся с Землей
4. Вывод данных о космической погоде
5. Выход

Пожалуйста введите номер функции: 1

Заголовок: LDN 1471: A Windblown Star Cavity
Дата: 2024-11-17
Ссылка на изображение: https://apod.nasa.gov/apod/image/2411/LDN1471_HubbleSchmidt_1024.jpg
Пожалуйста введите номер функции: 5


# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests

# Хранит данные о произведениях искусства и список работ автора
data = {}
atrhor_arts = []
author_name = []


def arts(page):
    # Загружает данные о произведениях искусства с указанной страницы.
    limit = 10
    url = f"https://api.artic.edu/api/v1/artworks?page={page}&limit={limit}"
    response = requests.get(url)

    # Проверка успешности запроса
    if response.status_code in [200, 201]:
        response = response.json()

        # Печать информации о текущей странице и произведениях
        print(f"\nСтраница номер {page}.")
        print(f"Количество произведений: {response['pagination']['limit']}.")
        print(f"Количество страниц: {response['pagination']['total_pages']}.")
        print(f"Ссылка на следующую страницу: {response['pagination']['next_url']}")
        if 'prev_url' in response['pagination']:
            print(f"Ссылка на предыдущую страницу: {response['pagination']['prev_url']}")

        print("\nСписок произведений:")
        count = 1
        for artwork in response['data']:
            print(f"{count}.{artwork['title']}")
            count += 1
        # Сохраняем данные для дальнейшего использования
        data.update(response)
    else:
        print(f"Ошибка запроса - {response.status_code}")


def author(name):
    #  Ищет произведения, созданные указанным автором.
    if not data:
        print("Сначала получите список произведений.")
        return
    else:
        # Сохраняем имя автора
        author_name.append(name)
        for artwork in data['data']:
            if name.lower() in artwork['artist_display'].lower():
                # Добавляем название произведения
                atrhor_arts.append(artwork['title'])

        print(f"Список произведений автора: {atrhor_arts}.\n")


def arts_names():
    # Выводит список произведений автора и запрашивает выбор произведения для получения подробной информации.
    print(f"Список произведений автора:")
    if atrhor_arts:
        count = 1
        for art in atrhor_arts:
            print(f"{count}.{art}")
            count += 1

        # Запрос выбора произведения
        while True:
            choice = int(input(f"Выберите любое произведение из интервала от 1 до {count - 1}: "))
            if choice > count - 1 or choice < 1:
                print("Выбранный номер не принадлежит интервалу, выберите правильный номер.\n")
            else:
                break

        # Печать подробной информации о выбранном произведении
        print("\nПодробная информация:")
        print(f"Название: {atrhor_arts[choice - 1]}.")
        print(f"Исполнитель: {author_name[0]}.")

        # Вывод дополнительной информации о произведении
        for date in data['data']:
            date_end = date['date_end']
            print(f"Дата написания: {date_end}.")
            print(f"Носитель: {date['medium_display']}")
            break
    else:
        print("Список произведений пуст.")
        return


def main():
    # Основная функция программы, которая управляет всеми функциями.
    print("Добро пожаловать в нашу функцию!")
    print(f"Функционал:",
          "1. Вывести список произведений по странице.",
          "2. Поиск произведений по автору в списке из функции 1.",
          "3. Вывод полной информации о произведении автора из списка функции 2.",
          "4. Выход",
          sep="\n")

    while True:
        choice = int(input("\nВыберите действие: "))
        if choice > 4 or choise < 1:
            print("Неправильный ввод.")
        else:
            if choice == 1:
                arts(page=int(input("Введите страницу: ")))
            elif choice == 2:
                author(name=input(
                    "Введите полное имя художника, к примеру 'Marcel Duchamp', иначе выдаст некорректный ответ: "))
            elif choice == 3:
                arts_names()
            elif choice == 4:
                print("Выход из программы.")
                break


# Запуск основной функции
main()


# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
import requests  # Библиотека для работы с api
from datetime import datetime  # Библиотека для работой с датой
crypto_portfolio = {'Баланс': {'usdt': 0, 'btc': 0, 'eth': 0}}  # Криптокошелек


def create_crypto():  # Функция управляет портфелем криптовалют

    print("Вы открыли портфель криптовалют.")
    print(crypto_portfolio)
    print("\nСписок доступных функций: ")
    print("1.Добавить монеты.", "2.Добавить криптовалюту.", "3.удалить криптовалюту.", "4.Общая стоимость.", sep="\n")
    print("5.Выход\n")

    while True:  # Пока наш цикл выполняется программа будет работать
        option = int(input("Выберете опцию: "))
        if option > 5 or option < 1:  # Ограничение для выбора опций
            print("\nВыберете одну из данных опций.")
            continue
        else:

            if option == 1:  # Опция 1 для добавления денег
                choice = input(f"Выберете кошелек {crypto_portfolio['Баланс']}: ").lower()
                if choice not in crypto_portfolio['Баланс']:
                    print("Данного кошелька не существует.")
                else:
                    while True:
                        money = int(input("Введи сколько криптовалюты ты хочешь добавить: "))
                        if money < 0:  # Проверка на возможный сбой при отрицательных значениях
                            print("Нельзя добавить отрицательные значения.")
                        else:
                            break
                    crypto_portfolio['Баланс'][choice] += money
                    print("Криптовалюта успешно добавлена.\n")

            elif option == 2:  # Опция 2 для добавления криптовалюты по ее короткому названию
                url = "https://api.coingecko.com/api/v3/coins/list"  # Список криптовалют
                response = requests.get(url)  # Отправляем get запрос данному api
                if response.status_code == 200 or response.status_code == 201:  # Проверка прошел ли запрос
                    response = response.json()  # Возращаем данные в виде словаря
                    choice = input("Введите криптовалюту: ")
                    for crypto in response:
                        if choice in crypto['symbol']:  # Проверяем есть ли криптовалюта в списке
                            if choice not in crypto_portfolio['Баланс']:  # Если ее нету то добавляем в портфолио
                                crypto_portfolio["Баланс"][choice] = 0
                                print("Криптовалюта успешна добавлена.\n")
                                break
                            else:
                                print("Данная криптовалюта уже имеется в портфолио.\n")
                                break
                    else:
                        print("Криптовалюта не найдена.")  # Если запрос не прошел выводим сообщение
                else:
                    print(f"Ошибка при подключении к api, код ошибки: {response.status_code}")
                    # Е

            elif option == 3:  # Опция 3 удаляет валюту из кошелька вместе с балансом
                choice = input("Введите криптовалюту, которую хотите удалить: ")
                if choice in crypto_portfolio['Баланс']:
                    del crypto_portfolio['Баланс'][choice]  # Удаляем криптовалюту если она в словаре
                    print("Криптовалюта успешно удалена.\n")
                else:
                    print("Данной криптовалюты нет в портфолио.\n")

            elif option == 4:  # Опция 4 подключается к апи исчитает суммарный баланс вложений
                print(crypto_portfolio)
                url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
                response = requests.get(url)  # Выполняем get запрос к апи
                if response.status_code == 200 or response.status_code == 201:  # Проверка запроса на ошибки
                    response = response.json()  # Если ошибок нет возращаем данные
                    total = 0   # Общий баланс
                    for money in response:
                        if money['symbol'] in crypto_portfolio['Баланс']:  # Если крипта в портфеле
                            symbol = money['symbol']   # Задаем переменную, хранящую символ
                            price = money['current_price']  # Задаем переменную, хранящую курс
                            total += price * crypto_portfolio['Баланс'][symbol]  # Считаем суммарный баланс
                            print(f"{symbol}: {price * crypto_portfolio['Баланс'][symbol]}")
                            # Выводим цену нашего актива из портфеля
                    print(f"Суммарный баланс: {total}\n")  # Выводим суммарный баланс
                else:
                    print(f"Ошибка при подключении к api, код ошибки: {response.status_code}")

            elif option == 5:
                break


def price_tracking():  # Функция отслеживает изменения цены в нашем портфеле и выводим самые сильные иизменения

    significant_price_change = {}
    print("\nИзменения цен в портфеле за последние 24ч: ")
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"

    response = requests.get(url)  # Также выполняем get запрос к api
    if response.status_code == 200 or response.status_code == 201:
        response = response.json()

        for price in response:
            symbol = price['symbol']
            if symbol in crypto_portfolio['Баланс']:  # Проверка того существует ли данная крипта в нашем портфеле
                print(f"{symbol}: {price['price_change_percentage_24h']:.3f}%")  # Для каждого актива выводим изменение
                significant_price_change[symbol] = price['price_change_percentage_24h']

        print(f"Значительно изменилась цена на {max(significant_price_change, key=significant_price_change.get)}\n")
        # Ищем максимальное изменение крипты в процентах

    else:
        print(f"Ошибка при подключении к api, код ошибки: {response.status_code}")


def historical_data():  # Функция для получения исторической справки о ценах
    id = input("Введите айди криптовалюты: ")
    start_date_str = input("Введите дату начала в виде YYYY-MM-DD: ")  # Начальный диапозон даты
    end_date_str = input("Введите дату окончания в виде YYYY-MM-DD: ")  # Конечный диапозон даты

    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    # Преобразуем в обьект datetime

    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())
    # Конвертируем дату в локальное время

    url = (f'https://api.coingecko.com/api/v3/coins/{id}/market_chart/range?vs_currency=usd&from='
           f'{start_timestamp}&to={end_timestamp}')

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        prices = data.get('prices', [])
        print(f"Цены на {id} с {start_date_str} по {end_date_str}:")
        for price in prices:
            date = datetime.fromtimestamp(price[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')  # Преобразуем секунды в дату
            price_value = price[1]
            print(f"Дата: {date}, Цена: ${price_value:.2f}")

    else:
        print(f"Ошибка при подключении к API, код ошибки: {response.status_code}")


def main():  # Главная функция, которая управляет остальными
    print("Список доступных команд:")
    print("1.Управление портфелем.", "2.Просмотр изменения цен.", "3.Просмотр историк  данных", "4.Выход.\n", sep="\n")

    while True:  # пока пользователь не выйдет программа будет работать
        choice = int(input("Введите команду: "))
        if choice == 1:
            create_crypto()  # Управление крипто кошельком (Она открывает а не создает крипто кошелек)
        elif choice == 2:
            price_tracking()  # Просмотр цен и баланса
        elif choice == 3:
            historical_data()  # Даты изменения цен в удобном формате
        elif choice == 4:
            print("Программа завершена!")
            exit(0)  # Выход из программы

        else:
            print("Введенной команды не существует.")


main()  # Вызываем главную функцию


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код